In [ ]:
# imports

import os
from datetime import date, datetime, timedelta
import cv2
import matplotlib.pyplot as plt
import pandas as pd

from util.constants import CLASS_LABELS, COLORS
from util.data import load_inference_data, output_to_arthur_format, download_model, \
    download_inference_dataset, download_reference_data, plot_color_key

In [ ]:
# download ultralytics/yolov5 library

from util.yololib import download_yolo_library

download_yolo_library()

## The Data

We'll download some image to run the model against. You can use a sample of images provided by Arthur, or download recent images from NASA.

### Load Sample Data

In [ ]:
# this will populate the 'api-data' folder with some sample images if none are present

download_inference_dataset()

### Viewing the Data

In [ ]:
SAMPLE_IMAGE_ID = "877390"

sample_image_path = f"./api-data/{SAMPLE_IMAGE_ID}/image.jpg"
sample_image = cv2.imread(sample_image_path)
plt.imshow(sample_image)

### Fetch New Mars Rover Images (Optional)

Next we'll fetch recent images from the [NASA Mars Rover Photos API](https://api.nasa.gov/#mars-rover-photos). You will need a NASA API Key, which you can obtain through the free signup at the top of the page.

In [ ]:
# Fill in your NASA API Key here to fetch fresh images

# os.environ['NASA_API_KEY'] = "<YOUR_API_KEY>"

In [ ]:
# this cell is tagged 'parameters' which allows us to parameterize it through Papermill

lookback_days = 15
mars_model_id = None

In [ ]:
new_image_ids = None

if 'NASA_API_KEY' in os.environ:
    from util import api

    # compute the specific days to query
    today = date.today()
    start_date = today - timedelta(days=lookback_days)

    # download the images
    new_image_ids = api.download_photos_in_day_range(start_date, today, camera="NAVCAM")

## The Model

Next we'll load the YOLO Model and our predict function. YOLO is a popular model that performs well at object detection tasks: drawing bounding boxes around different types of objects. We'll be applying this to the Mars images by detecting different types of terrain in the Martian landscape.

### Load Model

We'll download and load the PyTorch model from Arthur, pre-trained for the Martian Terrain object detection task

In [ ]:
# load the model

from predict import MarsPredictor

download_model()
model = MarsPredictor("./model/model_weights.pt")

### Prediction Example

In [ ]:
# fetch prediction from model
prediction = model.predict(sample_image_path, conf_thres=0.1)[0]

# plot bounding boxes on image
for bbox in prediction:
    x_start, y_start, x_end, y_end, confidence, class_idx = bbox
    class_label = CLASS_LABELS[int(class_idx)]
    color = COLORS[class_label]
    cv2.rectangle(sample_image, (int(x_start), int(y_start)), (int(x_end), int(y_end)),
                  color, 10)

plt.imshow(sample_image)

In [ ]:
# plot color key for box labels

plot_color_key()

## Arthur Integration

Next we'll onboard our model to the Arthur platform!

In [ ]:
# Arthur API Tokens are too long for some AWS environment variable specs, so load one from a file if present
#  (this section only applies to SageMaker deployments)
if "arthur-api-key.txt" in os.listdir():
    print("loading api key from file")
    with open("arthur-api-key.txt", 'r') as f:
        os.environ['ARTHUR_API_KEY'] = f.read().strip()

### Model Registration

In [1]:
# arthur imports

from arthurai import ArthurAI
from arthurai.common.constants import InputType, OutputType, Stage, ValueType, Enrichment

In [2]:
# create a connection to the Arthur API
# credentials are being passed to the client via environment variables

arthur = ArthurAI()

In [ ]:
# define an Arthur Model object

# define a unique model ID based on the current timestamp
partner_model_id = f"MarsRover-{datetime.now().strftime('%Y%m%d%H%M%S')}"

# plug in some basic metadata about our model: the input and output types as well as image dimensions
arthur_model = arthur.model(partner_model_id = partner_model_id,
                           display_name = "Martian Terrain",
                           input_type = InputType.Image,
                           output_type = OutputType.ObjectDetection,
                           pixel_width = 1024,
                           pixel_height = 1024)

# add our input image attribute
arthur_model.add_image_attribute("image")

# create prediction and ground truth columns for the detected objects
# each row of these attributes will contain a list of bounding boxes along with their associted class
predicted_attribute_name = "objects_detected"
ground_truth_attribute_name = "label"
arthur_model.add_object_detection_output_attributes(
    predicted_attribute_name, 
    ground_truth_attribute_name, 
    CLASS_LABELS)

# add additional metadata attributes that we'll supply with our inferences
arthur_model.add_attribute("martian_sol", stage=Stage.NonInputData,
                           value_type=ValueType.Integer)
arthur_model.add_attribute("image_id", stage=Stage.NonInputData,
                           value_type=ValueType.Integer)

arthur_model.review()

In [ ]:
# if we specified a model ID through papermill above, we'll only be sending inferences to it and can 
#  fetch it to overwrite the unsaved model we just created

if mars_model_id is None:
    arthur_model.save()
else:
    # (this section only applies to SageMaker deployments)
    arthur_model = arthur.get_model(mars_model_id)

### Add Reference Dataset

Next we'll download a reference dataset from Arthur. This includes a set of images and predictions that are used as a baseline for anomaly detection

In [ ]:
# if we just created the model (no previously-created model was passed in), also set the reference data

if mars_model_id is None:
    download_reference_data()
    arthur_model.set_reference_data(directory_path="./reference-data")

### Making Predictions

Finally we can use our loaded model and downloaded images to make predictions.

We'll then send these inferences to the Arthur platform to register them with the model we created.

In [ ]:
inference_df = load_inference_data(new_image_ids)
inference_df.head()

In [ ]:
predictions = model.predict(list(inference_df['image']), conf_thres=0.1)
inference_df['objects_detected'] = [output_to_arthur_format(x) for x in predictions]

In [ ]:
inference_df.rename(columns={'date': 'inference_timestamp'}, inplace=True)
inference_df['image_id'] = inference_df['image_id'].astype(int)

arthur_model.send_inferences(inference_df)

## Explore Your Data in the Arthur UI

Now that you've onboarded the model with Arthur, you can view the performance metrics and registered inferences in the platform. You can view the model's Mean Average Precision for the bounding boxes, detect drift through each image's Anomaly Score.

Use your rich visual interface to easily explore the images your model evaluated with the predicted bounding boxes and ground truth bounding boxes.

In [16]:
from urllib.parse import urlparse
from IPython.display import Markdown

api_url = urlparse(arthur.client.api_base_url)
dashboard_url = f"{api_url.scheme}://{api_url.netloc}"

Markdown(f"### Visit [your Arthur Dashboard]({dashboard_url})")


### Visit [your Arthur Dashboard](https://dev-v3.arthur.ai)